In [3]:
!pipenv install langchain_community
!pipenv install langchain
!pipenv install langchain-google-vertexai

Loading .env environment variables...
Courtesy Notice:
Pipenv found itself running within a virtual environment,  so it will 
automatically use that environment, instead of  creating its own for any 
project. You can set
PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and 
create  its own instead.
You can set PIPENV_VERBOSITY=-1 to suppress this warning.
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing langchain_community...
✔ Installation Succeeded
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing dependencies from Pipfile.lock (1866a3)...
All dependencies are now up-to-date!
Upgrading langchain_community in  dependencies.
Building requirements...
Resolving dependencies....
✔ Success! Locking packages...
⠧ Locking packages...
Building requirements...
Resolving dependencies....
✔ Success! Lock

In [4]:
from langchain_google_vertexai  import ChatVertexAI
import os

llm = ChatVertexAI(  
    model="gemini-2.0-flash-001",
)


/Users/itsyuimorii/.local/share/virtualenvs/ai-agent-capstone-zDg9zUJh/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


gcloud auth application-default login
2. set up google cloud project
gcloud config set project ai-agent-capstone

In [5]:
!pipenv install python-dotenv


Loading .env environment variables...
Courtesy Notice:
Pipenv found itself running within a virtual environment,  so it will 
automatically use that environment, instead of  creating its own for any 
project. You can set
PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and 
create  its own instead.
You can set PIPENV_VERBOSITY=-1 to suppress this warning.
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing python-dotenv...
✔ Installation Succeeded
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing dependencies from Pipfile.lock (1866a3)...
All dependencies are now up-to-date!
Upgrading python-dotenv in  dependencies.
Building requirements...
Resolving dependencies....
✔ Success! Locking packages...
⠴ Locking packages...
Building requirements...
Resolving dependencies....
✔ Success! Locking packages

In [6]:
!pipenv install google-search-result

Loading .env environment variables...
Courtesy Notice:
Pipenv found itself running within a virtual environment,  so it will 
automatically use that environment, instead of  creating its own for any 
project. You can set
PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and 
create  its own instead.
You can set PIPENV_VERBOSITY=-1 to suppress this warning.
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing google-search-result...
✔ Installation Succeeded
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing dependencies from Pipfile.lock (1866a3)...
All dependencies are now up-to-date!
Upgrading google-search-result in  dependencies.
Building requirements...
Resolving dependencies....
✘ Locking Failed!g packages...
⠴ Locking packages...CRITICAL:pipenv.patched.pip._internal.resolution.resolvelib.factor

In [ ]:

from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import tool
from dotenv import load_dotenv
import os
load_dotenv()

serpapi_api_key=os.environ["SERPAPI_API_KEY"]

@tool
def search_knowledge_base(query: str) -> str:
    """
    Use this tool only when you need to know real-time information or things you don't know!"""
    serp=SerpAPIWrapper()
    result = serp.run(query)
    return result

meimeishi_toolkit = []
search_toolkit = [search_knowledge_base]


from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate, SystemMessagePromptTemplate

def create_tool_agent(llm: ChatVertexAI, tools: list, system_prompt: str):
    system_prompt_template= PromptTemplate(
        template = system_prompt + """
        ONLY respond to the part of the query relevant to your purpose.
        IGNORE tasks you can not complete.
        USE the following context to answer your query 
        if available: \n {agent_history} \n
        """, 
        input_variables = ["agent_history"],
    )

    #define system message 
    system_message_prompt = SystemMessagePromptTemplate(prompt=system_prompt_template)
    
    prompt = ChatPromptTemplate.from_messages(
        [system_message_prompt,
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ]
    )
    agent = create_tool_calling_agent(llm, tools,prompt)
    executor= AgentExecutor(agent= agent , tools = tools , return_intermediate_steps = True, verbose = False)
    return executor 

from bot.naming_master_prompt import SYSTEM_PROMPT as meimeishi_system_prompt

system_prompt_template = """
    When the user requests current information that requires real-time data, such as:
    - Current events or breaking news
    - Recent statistics or data updates
    - Up-to-date prices, stock values, or market conditions
    - Weather forecasts or current conditions
    - Latest product releases or information
    - Trending topics or social media trends
    
    YOU SHOULD:
    Extract the key facts and important details from search results
    Organize the information in a clear, logical structure
    Focus only on information directly relevant to the user's question
    Provide context for data when necessary (e.g., compared to historical values)
    
    IMPORTANT: Present the information objectively and factually. When possible, include the date or time period the information refers to. If search results include contradictory information, acknowledge this and present multiple perspectives.
"""

In [ ]:

from langchain_core.messages import AIMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

meimeishi_agent = create_tool_agent(llm=llm, tools=meimeishi_toolkit, system_prompt=meimeishi_system_prompt)
search_agent = create_tool_agent(llm=llm, tools=search_toolkit, system_prompt =system_prompt_template)

system_prompt_template = PromptTemplate(
    template="""You are the supervisor agent, your responsibility is to gather information from the other agents and summarize agent history 
    in response to the original user query below 
    SUMMARIZE ALL THE OUTPUTS AND TOOLS USED in agent_history.
    the agent history is as follows: 
    \n{agent_history}\n""",
    input_variables=['agent_history'],
)

system_message_prompt = SystemMessagePromptTemplate(
    prompt=system_prompt_template)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message_prompt,
        MessagesPlaceholder(variable_name="messages")
    ]
)
comms_agent = (prompt | llm)

create the supervisor agent that will communicate the two available specialist agents for the user

In [19]:
from enum import Enum 
from pydantic import BaseModel 
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate, SystemMessagePromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent

members = ["Search_agent", "Meimeishi_agent"]

member_options = {member: member for member in members}

MemberEnum = Enum('MemberEnum', member_options)

class SupervisorOutput(BaseModel):
    #defaults to communication agent
    next: MemberEnum = MemberEnum.Search_agent

supervisor_system_prompt = """You are a Supervisor Agent responsible for orchestrating a team of specialized agents, including:

1. {members} - Each with unique capabilities and expertise

Your role is to:

1. Analyze the user's request and determine which agent is best suited to handle it
2. Route the conversation to the appropriate agent based on the request type
3. Monitor agent responses and progress
4. Decide when to transition between agents for complex tasks

Decision Guidelines:
- For Japanese naming requests: Route to the Meimei_agent (Japanese Naming Master)
- For real-time information needs: Route to the Search_agent

The conversation history and current state will guide your decision on which agent should act next.

Based on the current context, select one agent from: {options}

{format_instructions}
"""


supervisor_parser = JsonOutputParser(pydantic_object=SupervisorOutput)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_history")
       
    ]
).partial(options=str(members), members=", ".join(members), 
    format_instructions = supervisor_parser.get_format_instructions())
supervisor_chain = (
    prompt | llm |supervisor_parser
)


In [26]:
from langchain_core.messages import AIMessage
# For agents in the crew 
def crew_nodes(state, crew_member, name):
    #read the last message in the message history.
    input = {'messages': [state['messages'][-1]], 
                'agent_history' : state['agent_history']}
    result = crew_member.invoke(input)
    #add response to the agent history.
    return {"agent_history": [AIMessage(content= result["output"], additional_kwargs= {'intermediate_steps' : result['intermediate_steps']}, name=name)]}
def comms_node(state):
    #read the last message in the message history.
    input = {'messages': [state['messages'][-1]],'agent_history' : state['agent_history']}
    result = comms_agent.invoke(input)
    #respond back to the user.
    return {"messages": [result]}


In [ ]:
import operator
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str 
    
    agent_history: Annotated[Sequence[BaseMessage], operator.add]

workflow = StateGraph(AgentState)
math_node = functools.partial(crew_nodes, crew_member=math_agent, name="Calculator")
text_node = functools.partial(crew_nodes, crew_member=text_agent, name="TextEditor")
workflow.add_node("TextEditor", text_node)
workflow.add_node("Calculator", math_node)
workflow.add_node("Communicate", comms_node )
workflow.add_node("Supervisor", supervisor_chain)
#set it as entrypoint to the graph.
workflow.set_entry_point("Supervisor")

workflow.add_edge('TextEditor', "Supervisor") 
# add one edge for each of the tool agents
workflow.add_edge('Calculator', "Supervisor") 
# add one edge for each of the tool agents


In [ ]:
system_prompt_template = PromptTemplate(
    template = """ 
    When the user requests current information that's beyond your knowledge cutoff or requires real-time data, such as:
        - Current events, news, or trends
        - Recent statistics or data
        - Up-to-date prices, stock information, or market conditions
        - Weather forecasts or current conditions
        - Latest product releases or information
        - Recent name popularity statistics or current naming trends

        YOU SHOULD:
        1. Recognize when a search tool is needed for the query
        2. Use the search tool to find relevant, current information
        3. Extract the most important and accurate details from search results
        4. Organize the information in a clear, concise response
        5. Focus only on information directly relevant to the user's question
        6. Stay in character as Meimei Shi while delivering search results

        IMPORTANT: Search results should be integrated naturally into your responses. Present findings as knowledge you've discovered, not as "according to search results." Maintain your identity as Meimei Shi throughout.
    """
) 